In [ ]:
import sympy as sp
import numpy as np
import control as ct

In [ ]:
m1 = 0.269
m2 = 0.511
L1 = 0.307
L2 = 0.307
J1 = 1/12*m1*L1**2
J2 = 1/12*m2*L2**2
c1 = 0.1
c2 = 0.2
g = 9.82

# Transfer functions
T1 = s*(16*J_1*J_2*s + 4*J_1*l_2**2*m2*s + 16*J_2*c_1 + 4*J_2*l_1**2*m1*s + 16*J_2*l_1**2*m2*s + 4*c_1*l_2**2*m2 + l_1**2*l_2**2*m1*m2*s)/(16*J_1*J_2 + 4*J_1*l_2**2*m2 + 4*J_2*l_1**2*m1 + 16*J_2*l_1**2*m2 + l_1**2*l_2**2*m1*m2)
T2 = s*(16*J_1*J_2*s + 16*J_1*c_2 + 4*J_1*l_2**2*m2*s + 4*J_2*l_1**2*m1*s + 16*J_2*l_1**2*m2*s + 4*c_2*l_1**2*m1 + 16*c_2*l_1**2*m2 + l_1**2*l_2**2*m1*m2*s)/(16*J_1*J_2 + 4*J_1*l_2**2*m2 + 4*J_2*l_1**2*m1 + 16*J_2*l_1**2*m2 + l_1**2*l_2**2*m1*m2)